In [ ]:
import requests
import time
import pandas as pd
from io import StringIO
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union

# Setup

In [ ]:
SHAPEFILE_PATH = "YOUR-SHAPEFILE-PATH"

In [ ]:
PL_API_KEY = "YOUR-API-KEY"

In [ ]:
INIT_DATE="YYYY-MM-DD"
END_DATE="YYYY-MM-DD"

In [ ]:
PROD_ITEM_ID= "BIOMASS-PROXY_V4.0_10"#"LST-field_V1.0_20" #"SWC-field_V1.0_20"

# Verification
It checks to see if your organization has access/subscription for the Planetary Variables

In [ ]:
auth = requests.auth.HTTPBasicAuth(PL_API_KEY, "")
r = requests.get("https://api.planet.com/auth/v1/experimental/public/my/subscriptions", auth=auth)
print(r.status_code)
print(r.text[:400])

In [ ]:
BASE_URL = "https://api.planet.com/subscriptions/v1"
r = requests.get(BASE_URL, auth=auth)
print(r.status_code)
print(r.text[:600])

## Lists your access to products

In [ ]:
url = "https://api.planet.com/account/v1/my/products"

r = requests.get(
    url,
    headers={"Authorization": f"api-key {PL_API_KEY}", "Content-Type": "application/json"},
)
r.raise_for_status()
products = r.json()
for p in products.get("results", []):
    print(p.get("id"), p.get("name"), p.get("title"), "supports_reservation=", p.get("supports_reservation"))


## Filters by type (in this case Soil Water)

In [ ]:
results = products.get("results", [])

swc = [p for p in results if "SWC" in (p.get("name") or "") or "Soil Water" in (p.get("title") or "")]
for p in swc:
    print("ID:", p["id"])
    print("Name:", p["name"])
    print("Title:", p["title"])
    print("Resource IDs:", p.get("resource_ids", []))
    print("supports_reservation:", p.get("supports_reservation"))
    print("-"*60)

# Code Execution
## Helper function to read a SHAPEFILE and transform to an AOI

In [ ]:
def shapefile_to_clip_aoi(shp_path: str):
    gdf = gpd.read_file(shp_path)

    if gdf.empty:
        raise ValueError("Empty Shapefile.")

    # forces CRS in WGS84 (lon/lat)
    if gdf.crs is None:
        raise ValueError("Shapefile with CRS. Set it gdf.set_crs(...).")
    gdf = gdf.to_crs(epsg=4326)

    # unify in one single polygon
    geom = unary_union(gdf.geometry)

    # if MultiPolygon, get the bigger one or adapt
    if isinstance(geom, MultiPolygon):
        geom = max(list(geom.geoms), key=lambda p: p.area)

    if not isinstance(geom, Polygon):
        raise TypeError(f"Not a Polygon/MultiPolygon: {geom.geom_type}")

    # GeoJSON-like dict 
    geo = gpd.GeoSeries([geom], crs="EPSG:4326").__geo_interface__["features"][0]["geometry"]
    # geo já sai como {"type":"Polygon","coordinates":[...]}
    return geo

clip_aoi = shapefile_to_clip_aoi(SHAPEFILE_PATH)
clip_aoi

## Request a Subscription ID (based on PROD_ITEM_ID)

In [ ]:
headers = {
    "Authorization": f"api-key {PL_API_KEY}",
    "Content-Type": "application/json",
}

payload = {
  "name": "SWC time-series - AOI",
  "source": {
    "parameters": {
      "id": f"{PROD_ITEM_ID}",#"SWC-AMSR2-X_V5.0_1000",
      "start_time": f"{INIT_DATE}T00:00:00.0Z",
      "end_time":   f"{END_DATE}T23:59:59.0Z",
      "geometry": clip_aoi   # dict GeoJSON Polygon
    }
  }
  # no "delivery" => results-only
}

resp = requests.post(BASE_URL, json=payload, headers=headers)
print(resp.status_code, resp.text)
resp.raise_for_status()
subscription_id = resp.json()["id"]
subscription_id


## Process the subscription id request

In [ ]:
import time

results_url = f"https://api.planet.com/subscriptions/v1/{subscription_id}/results"
results = None
while True:
    resp = requests.get(results_url, auth=auth)
    resp.raise_for_status()
    data = resp.json()

    if data.get("results") and len(data["results"]) > 0:
        print("✅ Done!")
        results = (data["results"])
        break

    print("⏳ Still processing... waiting 60s")
    time.sleep(60)

## Parses the result

In [ ]:
print(results)

In [ ]:
import pandas as pd

if(results is not None):
    rows = []
    for r in results:
        props = r.get("properties", {})
        stats = props.get("statistics", [])
        if stats is not None:
            # transforma lista de stats em dict: {"mean": 0.274, "valid_percent": 100}
            stat_map = {s["name"]: s.get("value") for s in stats}
        
            rows.append({
                "datetime_utc": r.get("item_datetime"),
                "local_solar_time": props.get("local_solar_time"),
                "source_id": props.get("source_id"),
                "value (mean_swc | K |)": stat_map.get("mean"),
                "valid_percent": stat_map.get("valid_percent"),
                "result_id": r.get("id"),
                "status": r.get("status")
            })
    
    df = pd.DataFrame(rows)
    
    # tipos e ordenação
    df["datetime_utc"] = pd.to_datetime(df["datetime_utc"], utc=True)
    df = df.sort_values("datetime_utc").reset_index(drop=True)
    
    df


In [ ]:
df_valid = df[(df["valid_percent"] >= 0) & (df["value (mean_swc | K |)"].notna())].copy()
df_valid

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(df_valid["datetime_utc"], df_valid["mean_swc"])
plt.xlabel("Date (UTC)")
plt.ylabel("SWC mean")
plt.title("Soil Water Content (AOI mean)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
